1. Load the model Yeast8

In [10]:
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite

model = read_sbml_model('yeastGEM.xml')

2. Running model with Standard Medium

In [11]:
# optimize for biomass growth with Standard Medium

In [12]:
model.optimize()                               
print(model.objective.value)

0.08374778664999834


In [31]:
#Create reaction dictionary

In [13]:
xyodict = {1: "r_2105", # xylitol transport
           2: "r_1719", # D-xylose transport
           3: "r_1093", # xylose reductase
           4: "r_1092"  # xylitol dehydrogenase
          }

In [14]:
#Setting different medium conditions. The elements in the different mediums can be displayed replacing the pertinent number in the bottom code

In [99]:
medium_glu = model.medium  # 100% glucose as carbon source in the medium

with model: 
    medium_xyl['r_1714'] = 2.0

medium_xyl = model.medium  # 100% xylose as carbon source in the medium
with model: 
    medium_xyl['r_1714'] = 0.0
    medium_xyl['r_1718'] = 2.0

medium_50 = model.medium # 50% glucose and 50% xylose as carbon sources in the medium
with model: 
    medium_50['r_1714'] = 1.0
    medium_50['r_1718'] = 1.0
    
medium_glu15 = model.medium # 75% glucose and 25% xylose as carbon sources in the medium
with model: 
    medium_glu15['r_1714'] = 1.5
    medium_glu15['r_1718'] = 0.5
    
medium_xyl15 = model.medium # 25% glucose and 75% xylose as carbon sources in the medium
with model: 
    medium_xyl15['r_1714'] = 0.5
    medium_xyl15['r_1718'] = 1.5
    
medium_glu12 = model.medium # 60% glucose and 40% xylose as carbon sources in the medium
with model: 
    medium_glu12['r_1714'] = 1.2
    medium_glu12['r_1718'] = 0.8

medium_xyl12 = model.medium # 40% glucose and 60% xylose as carbon sources in the medium
with model: 
    medium_xyl12['r_1714'] = 0.8
    medium_xyl12['r_1718'] = 1.2

for element in medium_glu.keys():
    print(model.reactions.get_by_id(element).name, model.reactions.get_by_id(element), medium_glu[element])

ammonium exchange r_1654: s_0420[e] <=>  1000.0
D-glucose exchange r_1714: s_0565[e] <=>  1.0
H+ exchange r_1832: s_0796[e] <=>  1000.0
iron(2+) exchange r_1861: s_0925[e] <=>  1000.0
oxygen exchange r_1992: s_1277[e] <=>  1000.0
phosphate exchange r_2005: s_1324[e] <=>  1000.0
potassium exchange r_2020: s_1374[e] <=>  1000.0
sodium exchange r_2049: s_1438[e] <=>  1000.0
sulphate exchange r_2060: s_1468[e] <=>  1000.0
water exchange r_2100: s_0805[e] <=>  1000.0
chloride exchange r_4593: s_4200[e] <=>  1000.0
Cu2(+) exchange r_4594: s_4201[e] <=>  1000.0
Mn(2+) exchange r_4595: s_4202[e] <=>  1000.0
Zn(2+) exchange r_4596: s_4203[e] <=>  1000.0
Mg(2+) exchange r_4597: s_4204[e] <=>  1000.0
Ca(2+) exchange r_4600: s_4199[e] <=>  1000.0


3. Run the model with the selected medium. In this model there are already included a combination of the different mutations that have been proven to work individually with promise to improve xylitol production. This include upregulation of a xylose transporter form the medium to inside of the cell, an upregulation of the xylose reductase, for a higher xylitol conversion, a knock out of the gene XDH in the PPP pathway to prevent xylitol to keep going down the pathway, and an upregulation of the xylitol transporter. In the code below, we can combine these modifications with mediums with different composition, using alternative proportions of the two selected carbon sources: glucose and xylose. The idea behind this experiment is to determine the best conditions for xylitol production while still having cell growth. Also, all of the designed mutations are present and depicted. The idea behind it is to find the best combinations. 

In [98]:
with model:
    # model.medium = medium_glu  # 100% glucose as carbon source in the medium
    # model.medium = medium_xyl # 100% xylose as carbon source in the medium
    # model.medium = medium_glu15 # 75% glucose and 25% xylose as carbon sources in the medium
    model.medium = medium_50 # 50% glucose and 50% xylose as carbon sources in the medium
    # model.medium = medium_xyl15 # 25% glucose and 75% xylose as carbon sources in the medium
    # model.medium = medium_glu12 # 60% glucose and 40% xylose as carbon sources in the medium
    # model.medium = medium_xyl12 # 40% glucose and 60% xylose as carbon sources in the medium
    ## regulations
    model.reactions.r_1719.bounds = (0,10) # regulation for xylose transporter
    model.reactions.r_1093.bounds = (0,10) # regulation for xylose reductase
    model.genes.YLR070C.knock_out() # knockout for xylitol dehydrogenase
    model.reactions.r_2104.bounds = (0,2) # regulation for xylitol transporter
    model.reactions.r_0889.bounds = (0,10) # 6-phosphogluconate dehydrogenase
    model.reactions.r_0466.bounds = (0,10) # Glucose-6-phosphate dehydrogenase

    
    ## solve model
    model.objective = model.reactions.r_2104 # objective is maximum xylitol production
    # model.objective = model.reactions.r_1093 # objective is maximum xylose reductase
    xylitol_production = model.optimize().objective_value
    print(model.summary())
    for key in xyodict.keys():
        print(model.reactions.get_by_id(xyodict[key]).name, model.reactions.get_by_id(xyodict[key]).flux)
    print(model.reactions.r_2111.name, model.reactions.r_2111.flux)
    print(model.reactions.r_1714.name, model.reactions.r_1714.flux)
    print(model.reactions.r_1166.name, model.reactions.r_1166.flux)
    print("Max.xylitol production [mmol gDW^-1]:", xylitol_production)
    print("Theoretical max.yield [mmol-xylitol / mmol-xylose]:", xylitol_production/(-1*model.reactions.r_1718.flux))

Objective
1.0 r_2104 = 0.9999999999999998

Uptake
------
Metabolite Reaction   Flux  C-Number  C-Flux
 s_0565[e]   r_1714 0.2378         6  22.20%
 s_0579[e]   r_1718      1         5  77.80%
 s_1277[e]   r_1992 0.1545         0   0.00%
 s_0805[e]   r_2100 0.1911         0   0.00%

Secretion
---------
Metabolite Reaction    Flux  C-Number  C-Flux
 s_0458[e]   r_1672    -0.5         1   7.78%
 s_0796[e]   r_1832 -0.3089         0   0.00%
 s_1400[e]   r_2033 -0.3089         3  14.42%
 s_1567[e]   r_2104      -1         5  77.80%

xylitol transport -0.9999999999999998
D-xylose transport 1.0
xylose reductase 1.0
xylitol dehydrogenase (D-xyulose-forming) 0.0
growth 0.0
D-glucose exchange -0.23779052662547806
glucose transport 0.23779052662547806
Max.xylitol production [mmol gDW^-1]: 0.9999999999999998
Theoretical max.yield [mmol-xylitol / mmol-xylose]: 0.9999999999999998


We can see how when the model is run with the standard medium medium_glu (no xylose is present) the xylitol transporter, xylose transporter, xylose reductase (XR) and xylitol dehydrogenase (XDH) have flux equal to 0. That means that with out Xylose in the media all the pathway is not active. We can also stablish that when the medium only has xylose, reaction r_2111 associated to cell growth is 0, suggesting that in this medium the cells even though they are able to produce xylitol, they are not able to multiply with the selected mutations. In fact the pure xylose medium with all of the mutations included results in the inhabilty of the cell to survive, while when we combine the mutants with a hybrid medium of xylose/glucose they are able to grow and produce xylitol in at a better rate than the unmodified strain, depending to the media composition. The code above allows the exploration of combinations of mutations and media conditions. The best observed results vary depending the objective you want to achieve. When all of the modifications designed for improving the xylitol production are in place (upregulation of the xylose transporter, upregulation of the xylose reductase (XR), knock out of the xylitol reductase (XR), and up regulation of the xylitol transporter), the best results for growth (r_2111) in combination with maximun xylitol production (r_2104) vary depending on what is prioritariced. The presence of glucose is directly related to an increase in growth, while, with all of this mutations in place, all of the xyolse intake seems to go towards xylitol production and exportation. This corresponds to previous assestments seen in the literature [12], where the possibility is stated of a bioconversion of up to 99%.

The growth increases to 0.154 when the growth media insted of having glucose have both xylose and glucose in equal proportions. Now it can also be seen how the  the Xylose transport, the XR and the XDH are active, however there is not Xylitol transport. 